In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/keywords/Keyword_Medical.json
/kaggle/input/keywords/Keyword_Canon.json
/kaggle/input/keywords/Keyword_Creative.json
/kaggle/input/keywords/Keyword_Apex.json
/kaggle/input/keywords/Keyword_Apex 1.json
/kaggle/input/keywords/Keyword_Nokia.json
/kaggle/input/keywords/Keyword_News.json
/kaggle/input/keywords/Keyword_Nikon.json
/kaggle/input/zstikg/NewsConcept Data-set.json
/kaggle/input/zstikg/DVD playerData-set.json
/kaggle/input/zstikg/MedicalConcept Data-set.json
/kaggle/input/zstikg/Cellular phone Data-set.json
/kaggle/input/zstikg/Digital camera2 Data-set.json
/kaggle/input/zstikg/Mp3 playerData-set.json
/kaggle/input/zstikg/Digital camera1 Data-set.json


In [12]:
# ============================================================================
# PART 1: INSTALLATION AND SETUP
# ============================================================================

# Install required packages
!pip install dspy-ai huggingface_hub networkx sentence-transformers pandas --quiet

print("✓ Packages installed successfully!")

✓ Packages installed successfully!


In [13]:
# ============================================================================
# PART 1: IMPORTS
# ============================================================================

import json
import networkx as nx
from typing import List, Dict, Set, Tuple
import dspy
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import defaultdict
import pandas as pd
import os
from huggingface_hub import InferenceClient

print("✓ All imports successful!")

✓ All imports successful!


In [15]:
# ============================================================================
# STEP 2: CONFIGURE HUGGING FACE WITH DSPY.LM (PROPER WAY)
# ============================================================================

os.environ['HUGGINGFACE_API_KEY'] = 'KEY'

# Use DSPy's built-in LM class with the correct prefix
lm = dspy.LM(
    model='huggingface/meta-llama/Llama-3.1-8B-Instruct',
    api_key=os.environ['HUGGINGFACE_API_KEY'],
    max_tokens=12000,
    temperature=0.3
)

dspy.settings.configure(lm=lm)
print("✓ Llama-3.1-8B-Instruct configured successfully!")

✓ Llama-3.1-8B-Instruct configured successfully!


In [16]:
# ============================================================================
# PART 1: DATA LOADING FUNCTION
# ============================================================================

def load_json_data(filepath: str) -> List[Dict]:
    """
    Load JSON data from file
    
    Expected format:
    [
        {
            "Article Title": [],
            "Article Text": "text here...",
            "Concept": ["concept1", "concept2"]
        },
        ...
    ]
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"✓ Loaded {len(data)} documents from {filepath}")
    return data

# Test with sample data
sample_data = [
    {
        "Article Title": [],
        "Article Text": "excellent phone, excellent service.",
        "Concept": []
    },
    {
        "Article Title": [],
        "Article Text": "i am a business user who heavily depend on mobile service.",
        "Concept": ["service"]
    }
]

print("✓ Sample data ready for testing")
print(f"  Sample has {len(sample_data)} documents")

✓ Sample data ready for testing
  Sample has 2 documents


In [17]:
# ============================================================================
# LOAD ONE SPECIFIC DATASET
# ============================================================================

# Choose which dataset you want to work with:
# Option 1: Cellular phone
filepath = '/kaggle/input/zstikg/Mp3 playerData-set.json'

# Option 2: News
# filepath = '/kaggle/input/zsltikg/NewsConcept Data-set.json'

# Option 3: Medical
# filepath = '/kaggle/input/zsltikg/MedicalConcept Data-set.json'

# Load the data
data = load_json_data(filepath)

print(f"✓ Loaded {len(data)} documents")
print(f"\nFirst document preview:")
print(f"  Keys: {list(data[0].keys())}")
print(f"  Text: {data[0].get('Article Text', '')}...")
print(f"  Concepts: {data[0].get('Concept', [])}")

✓ Loaded 1811 documents from /kaggle/input/zstikg/Mp3 playerData-set.json
✓ Loaded 1811 documents

First document preview:
  Keys: ['Article Title', 'Article Text', 'Concept']
  Text:  a quick update to the new zen nx ? 
...
  Concepts: []


In [18]:
# ============================================================================
# PROCESS YOUR ALL THE ARTICLES
# ============================================================================

print("=" * 80)
print("PROCESSING ALL THE ARTICLES")
print("=" * 80)

# data is now correctly loaded as a list of dicts
individual_articles = []

for idx, item in enumerate(data):
    article = {
        'id': idx,
        'Article Text': item['Article Text'],
        #'Concept': item['Concept'] if item['Concept'] else []
    }
    individual_articles.append(article)

print(f"\n✓ Created list of {len(individual_articles)} individual articles")

# Show first 3
print(f"\nFirst 3 articles:")
print("-" * 80)
for i in range(min(3, len(individual_articles))):
    article = individual_articles[i]
    print(f"\nArticle {article['id']}:")
    print(f"  Text: {article['Article Text'][:80]}...")
    #print(f"  Concepts: {article['Concept']}")

PROCESSING ALL THE ARTICLES

✓ Created list of 1811 individual articles

First 3 articles:
--------------------------------------------------------------------------------

Article 0:
  Text:  a quick update to the new zen nx ? 
...

Article 1:
  Text: this is an edited review , now that i have had time to use the device . 
...

Article 2:
  Text: while , there are flaws with the machine , the xtra gets five stars because of i...


In [19]:
# ============================================================================
# STEP 3: DEFINE SIGNATURES
# ============================================================================

class EntityExtractor(dspy.Signature):
    """Extract key entities from the given text. Extracted entities are nouns, 
    verbs, or adjectives, particularly regarding sentiment. This is for an 
    extraction task, please be thorough and accurate to the reference text.
    
    Return ONLY a valid JSON list format: ["entity1", "entity2", "entity3"]
    """
    
    text = dspy.InputField(desc="The text to extract entities from")
    entities = dspy.OutputField(desc="List of extracted entities in JSON format")

class RelationExtractor(dspy.Signature):
    """Extract subject-predicate-object triples from the assistant message. 
    A predicate (1-3 words) defines the relationship between the subject and 
    object. Relationship may be fact or sentiment based on assistant's message. 
    Subject and object are entities. Entities provided are from the assistant 
    message and prior conversation history, though you may not need all of them. 
    This is for an extraction task, please be thorough, accurate, and faithful 
    to the reference text.
    
    Return ONLY valid JSON format: [["subject1", "predicate1", "object1"], ["subject2", "predicate2", "object2"]]
    """
    
    text = dspy.InputField(desc="The text to extract relations from")
    entities = dspy.InputField(desc="List of available entities")
    triples = dspy.OutputField(desc="List of [subject, predicate, object] triples in JSON format")

print("✓ Signatures defined")

✓ Signatures defined


In [20]:
# ============================================================================
# STEP 4: CREATE ENTITY EXTRACTOR
# ============================================================================

class ExtractEntities(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extract = dspy.ChainOfThought(EntityExtractor)
    
    def forward(self, text: str) -> List[str]:
        if not text or len(text.strip()) < 3:
            return []
            
        result = self.extract(text=text)
        
        try:
            entities_text = result.entities.strip()
            
            if '[' in entities_text and ']' in entities_text:
                start = entities_text.find('[')
                end = entities_text.rfind(']') + 1
                entities_text = entities_text[start:end]
            
            entities = json.loads(entities_text)
            
            if isinstance(entities, list):
                return [str(e).lower().strip() for e in entities if e and len(str(e).strip()) > 1]
            return []
            
        except:
            try:
                entities_text = result.entities.strip()
                if entities_text.startswith('['):
                    entities_text = entities_text[1:]
                if entities_text.endswith(']'):
                    entities_text = entities_text[:-1]
                
                entities = []
                for item in entities_text.split(','):
                    item = item.strip(' "\'\n\t')
                    if item and len(item) > 1:
                        entities.append(item.lower())
                
                return entities[:50]
            except:
                return []

entity_extractor = ExtractEntities()
print("✓ Entity Extractor created")

✓ Entity Extractor created


In [21]:
# ============================================================================
# STEP 5: CREATE RELATION EXTRACTOR
# ============================================================================

class ExtractRelations(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extract = dspy.ChainOfThought(RelationExtractor)
    
    def forward(self, text: str, entities: List[str]) -> List[Tuple[str, str, str]]:
        if not entities or not text:
            return []
        
        entities_subset = entities[:30]
        entities_str = json.dumps(entities_subset)
        
        result = self.extract(text=text, entities=entities_str)
        
        try:
            triples_text = result.triples.strip()
            
            if '[' in triples_text and ']' in triples_text:
                start = triples_text.find('[')
                end = triples_text.rfind(']') + 1
                triples_text = triples_text[start:end]
            
            triples = json.loads(triples_text)
            
            normalized_triples = []
            for triple in triples:
                if isinstance(triple, (list, tuple)) and len(triple) == 3:
                    s, p, o = triple
                    s = str(s).lower().strip()
                    p = str(p).lower().strip()
                    o = str(o).lower().strip()
                    
                    if s and p and o and s != o:
                        normalized_triples.append((s, p, o))
            
            return normalized_triples
            
        except Exception as e:
            return []

relation_extractor = ExtractRelations()
print("✓ Relation Extractor created")

✓ Relation Extractor created


In [22]:
# ============================================================================
# CLUSTERING SIGNATURES (DEFINE FIRST!)
# ============================================================================

class ClusterValidator(dspy.Signature):
    """Verify if these entities belong in the same cluster.
    A cluster should contain entities that are the same in meaning, with different:
    - tenses, plural forms, stem forms, upper/lower cases
    Or entities with close semantic meanings.
    
    Return ONLY valid JSON format: ["entity1", "entity2", "entity3"]
    Return only entities you are confident belong together.
    If not confident, return empty list [].
    """
    
    entities = dspy.InputField(desc="Entities to validate")
    valid_cluster = dspy.OutputField(desc="Validated cluster in JSON format")

print("✓ ClusterValidator Signature defined")

✓ ClusterValidator Signature defined


In [23]:
# ============================================================================
# SEMANTIC SIMILARITY CLUSTERING (FROM PAPER)
# ============================================================================

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class SemanticEntityClustering(dspy.Module):
    def __init__(self, similarity_threshold=0.75):
        super().__init__()
        self.validator = dspy.ChainOfThought(ClusterValidator)
        
        # Load embedding model (same as paper)
        print("Loading sentence transformer model...")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        print("✓ Model loaded")
        
        self.similarity_threshold = similarity_threshold
    
    def _parse_cluster(self, text: str) -> List[str]:
        """Parse cluster from LLM response"""
        try:
            text = text.strip()
            if '[' in text and ']' in text:
                start = text.find('[')
                end = text.rfind(']') + 1
                text = text[start:end]
            
            cluster = json.loads(text)
            if isinstance(cluster, list):
                return [str(e).lower().strip() for e in cluster if e]
            return []
        except:
            return []
    
    def _get_semantic_clusters(self, entities: List[str]) -> List[List[str]]:
        """Group entities by semantic similarity using embeddings"""
        
        if len(entities) == 0:
            return []
        
        # Get embeddings for all entities
        embeddings = self.model.encode(entities)
        
        # Compute pairwise cosine similarity
        similarity_matrix = cosine_similarity(embeddings)
        
        # Find clusters using similarity threshold
        clusters = []
        remaining = set(range(len(entities)))
        
        for i in range(len(entities)):
            if i not in remaining:
                continue
            
            # Find all entities similar to this one
            cluster_indices = [i]
            remaining.discard(i)
            
            for j in range(i + 1, len(entities)):
                if j not in remaining:
                    continue
                
                # Check if similar enough
                if similarity_matrix[i][j] >= self.similarity_threshold:
                    cluster_indices.append(j)
                    remaining.discard(j)
            
            # Convert indices to entity names
            cluster = [entities[idx] for idx in cluster_indices]
            
            # Only keep clusters with 2-4 entities
            if 2 <= len(cluster) <= 4:
                clusters.append(cluster)
            elif len(cluster) == 1:
                # Keep singletons for later
                pass
        
        return clusters
    
    def forward(self, entities: List[str]) -> Dict[str, List[str]]:
        """Semantic clustering with LLM validation"""
        
        print(f"Starting semantic clustering with {len(entities)} entities...")
        print(f"  Similarity threshold: {self.similarity_threshold}")
        
        # Remove duplicates
        unique_entities = list(set(entities))
        
        # Step 1: Find semantic clusters using embeddings
        print("  Computing semantic similarities...")
        potential_clusters = self._get_semantic_clusters(unique_entities)
        
        print(f"  Found {len(potential_clusters)} potential clusters")
        
        # Step 2: Validate with LLM
        validated_clusters = {}
        cluster_id = 0
        clustered_entities = set()
        
        for cluster in potential_clusters:
            try:
                # Ask LLM to validate
                validation = self.validator(entities=json.dumps(cluster))
                validated = self._parse_cluster(validation.valid_cluster)
                
                if validated and len(validated) >= 2:
                    cluster_label = validated[0]
                    validated_clusters[cluster_label] = validated
                    
                    for entity in validated:
                        clustered_entities.add(entity)
                    
                    print(f"  ✓ Cluster {cluster_id}: {validated}")
                    cluster_id += 1
                else:
                    # LLM rejected - add as singletons
                    for entity in cluster:
                        if entity not in clustered_entities:
                            validated_clusters[entity] = [entity]
                            clustered_entities.add(entity)
            except:
                # Error - add as singletons
                for entity in cluster:
                    if entity not in clustered_entities:
                        validated_clusters[entity] = [entity]
                        clustered_entities.add(entity)
        
        # Step 3: Add all remaining entities as singletons
        for entity in unique_entities:
            if entity not in clustered_entities:
                validated_clusters[entity] = [entity]
        
        multi = sum(1 for v in validated_clusters.values() if len(v) > 1)
        print(f"✓ Semantic clustering complete: {len(validated_clusters)} total clusters")
        print(f"  Multi-entity clusters: {multi}")
        print(f"  Singleton entities: {len(validated_clusters) - multi}")
        
        return validated_clusters

# Create semantic clusterer with different thresholds
entity_clusterer_semantic = SemanticEntityClustering(similarity_threshold=0.75)
print("\n✓ Semantic Entity Clustering Module created")

Loading sentence transformer model...


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

✓ Model loaded

✓ Semantic Entity Clustering Module created


In [24]:
# ============================================================================
# FINAL KGGEN WITH SEMANTIC CLUSTERING
# ============================================================================

class KGGenSemantic:
    def __init__(self, similarity_threshold=0.75):
        self.entity_extractor = entity_extractor
        self.relation_extractor = relation_extractor
        self.entity_clusterer = SemanticEntityClustering(similarity_threshold=similarity_threshold)
        self.graph = nx.DiGraph()
        self.entity_clusters = {}
        
    def generate_from_json(self, json_data: List[Dict], max_docs: int = None) -> nx.DiGraph:
        """Generate KG from JSON dataset"""
        all_entities = set()
        all_relations = []
        
        if max_docs:
            json_data = json_data[:max_docs]
        
        print(f"Processing {len(json_data)} documents...")
        print("=" * 80)
        
        for idx, item in enumerate(json_data):
            text = item.get('Article Text', '')
            concepts = item.get('Concept', [])
            
            if not text or len(text.strip()) < 5:
                continue
            
            try:
                # Extract entities
                entities = self.entity_extractor(text)
                all_entities.update(entities)
                
                # Add concepts
                #for concept in concepts:
                    #if concept and isinstance(concept, str):
                        #all_entities.add(concept.lower().strip())
                
                # Extract relations
                relations = self.relation_extractor(text, list(all_entities))
                all_relations.extend(relations)
                
                if (idx + 1) % 20 == 0:
                    print(f"  {idx + 1}/{len(json_data)} docs | {len(all_entities)} entities | {len(all_relations)} relations")
                    
            except Exception as e:
                continue
        
        print(f"\n✓ Extraction complete!")
        print(f"  Total entities: {len(all_entities)}")
        print(f"  Total relations: {len(all_relations)}")
        
        # Build graph
        for subj, pred, obj in all_relations:
            self.graph.add_edge(subj, obj, relation=pred)
        
        print(f"  Graph nodes: {len(self.graph.nodes())}")
        print(f"  Graph edges: {len(self.graph.edges())}")
        
        return self.graph
    
    def cluster_entities(self):
        """Semantic clustering with embeddings"""
        nodes = list(self.graph.nodes())
        
        if len(nodes) == 0:
            print("No nodes to cluster!")
            return self.graph
        
        print(f"\n{'='*80}")
        print(f"SEMANTIC CLUSTERING: {len(nodes)} ENTITIES")
        print(f"{'='*80}")
        
        self.entity_clusters = self.entity_clusterer(nodes)
        
        # Map entities
        entity_mapping = {}
        for cluster_label, cluster_entities in self.entity_clusters.items():
            for entity in cluster_entities:
                entity_mapping[entity] = cluster_label
        
        # Rebuild graph
        new_graph = nx.DiGraph()
        for u, v, data in self.graph.edges(data=True):
            new_u = entity_mapping.get(u, u)
            new_v = entity_mapping.get(v, v)
            relation = data.get('relation', 'related_to')
            
            if new_u == new_v:
                continue
            
            if not new_graph.has_edge(new_u, new_v):
                new_graph.add_edge(new_u, new_v, relation=relation)
        
        self.graph = new_graph
        
        print(f"\n✓ Clustering complete!")
        print(f"  Final nodes: {len(self.graph.nodes())}")
        print(f"  Final edges: {len(self.graph.edges())}")
        
        return self.graph
    
    def save_graph(self, filepath: str):
        data = nx.node_link_data(self.graph)
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
        print(f"✓ Saved to {filepath}")
    
    def export_triples(self, filepath: str):
        triples = []
        for u, v, data in self.graph.edges(data=True):
            triples.append({
                'subject': u,
                'predicate': data.get('relation', 'related_to'),
                'object': v
            })
        import pandas as pd
        df = pd.DataFrame(triples)
        df.to_csv(filepath, index=False)
        print(f"✓ Exported to {filepath}")

# Initialize semantic KGGen (threshold 0.75 = balanced)
kg_gen_semantic = KGGenSemantic(similarity_threshold=0.75)
print("\n✓ KGGen with Semantic Clustering (Embeddings + LLM) created!")

Loading sentence transformer model...
✓ Model loaded

✓ KGGen with Semantic Clustering (Embeddings + LLM) created!


In [25]:
# ============================================================================
# GENERATE KG FOR ALL THE ARTICLES
# ============================================================================

print("\n" + "=" * 80)
print("GENERATING KNOWLEDGE GRAPHS FOR ALL THE ARTICLES")
print("=" * 80)

all_article_kgs = []

total = len(individual_articles)
print(f"\nProcessing {total} articles...\n")

for idx, article in enumerate(individual_articles):
    article_id = article['id']
    text = article['Article Text']
    #concepts = article['Concept']
    
    # Skip empty articles
    if not text or len(text.strip()) < 5:
        all_article_kgs.append({
            'id': article_id,
            'text': text,
            #'concepts': concepts,
            'graph': nx.DiGraph(),
            'entities': [],
            'relations': [],
            'num_nodes': 0,
            'num_edges': 0
        })
        continue
    
    try:
        # Extract entities for THIS article
        entities = entity_extractor(text)
        
        # Add original concepts as entities
        #for concept in concepts:
            #if concept and isinstance(concept, str):
                #entities.append(concept.lower().strip())
        
        entities = list(set(entities))  # Remove duplicates
        
        # Extract relations for THIS article
        if entities:
            relations = relation_extractor(text, entities)
        else:
            relations = []
        
        # Build graph for THIS article
        graph = nx.DiGraph()
        for subj, pred, obj in relations:
            graph.add_edge(subj, obj, relation=pred)
        
        # Store everything for this article
        all_article_kgs.append({
            'id': article_id,
            'text': text,
            #'concepts': concepts,
            'graph': graph,
            'entities': entities,
            'relations': relations,
            'num_nodes': len(graph.nodes()),
            'num_edges': len(graph.edges())
        })
        
        # Progress update
        if (idx + 1) % 50 == 0:
            print(f"✓ Processed {idx + 1}/{total} articles...")
        
    except Exception as e:
        print(f"✗ Article {article_id}: Error - {str(e)}")
        all_article_kgs.append({
            'id': article_id,
            'text': text,
            #'concepts': concepts,
            'graph': nx.DiGraph(),
            'entities': [],
            'relations': [],
            'num_nodes': 0,
            'num_edges': 0
        })

print(f"\n" + "=" * 80)
print("✓ KNOWLEDGE GRAPH GENERATION COMPLETE!")
print("=" * 80)
print(f"Total articles processed: {len(all_article_kgs)}")
print(f"Articles with graphs: {sum(1 for kg in all_article_kgs if kg['num_edges'] > 0)}")
print(f"Articles without graphs: {sum(1 for kg in all_article_kgs if kg['num_edges'] == 0)}")

# Statistics
total_entities = sum(len(kg['entities']) for kg in all_article_kgs)
total_relations = sum(len(kg['relations']) for kg in all_article_kgs)

print(f"\nTotal entities extracted: {total_entities}")
print(f"Total relations extracted: {total_relations}")

with_graphs = [kg for kg in all_article_kgs if kg['num_edges'] > 0]
if with_graphs:
    avg_nodes = sum(kg['num_nodes'] for kg in with_graphs) / len(with_graphs)
    avg_edges = sum(kg['num_edges'] for kg in with_graphs) / len(with_graphs)
    #print(f"\nAverage nodes per KG: {avg_nodes:.2f}")
    #print(f"Average edges per KG: {avg_edges:.2f}")


GENERATING KNOWLEDGE GRAPHS FOR ALL THE ARTICLES

Processing 1811 articles...

✓ Processed 50/1811 articles...
✓ Processed 100/1811 articles...
✓ Processed 150/1811 articles...
✓ Processed 200/1811 articles...
✓ Processed 250/1811 articles...
✓ Processed 300/1811 articles...
✓ Processed 350/1811 articles...
✓ Processed 400/1811 articles...
✓ Processed 450/1811 articles...
✓ Processed 500/1811 articles...
✓ Processed 550/1811 articles...
✓ Processed 600/1811 articles...
✓ Processed 650/1811 articles...
✓ Processed 700/1811 articles...
✓ Processed 750/1811 articles...
✓ Processed 800/1811 articles...
✓ Processed 850/1811 articles...
✓ Processed 900/1811 articles...
✓ Processed 950/1811 articles...
✓ Processed 1000/1811 articles...
✓ Processed 1050/1811 articles...
✓ Processed 1100/1811 articles...
✓ Processed 1150/1811 articles...
✓ Processed 1200/1811 articles...
✓ Processed 1250/1811 articles...
✓ Processed 1300/1811 articles...
✓ Processed 1350/1811 articles...
✓ Processed 1400/1811 

In [35]:
# ============================================================================
# STEP 1: IMPORTS
# ============================================================================

import json
import networkx as nx
from typing import List, Dict, Set, Tuple
import dspy
from dspy.signatures import Signature
from dspy import OutputField, InputField
import pandas as pd
import os
from collections import Counter
import numpy as np

print("✓ All imports successful!")

✓ All imports successful!


In [36]:
# ============================================================================
# SAVE KNOWLEDGE GRAPHS TO JSON FILE
# ============================================================================

import json
import networkx as nx

def save_knowledge_graphs(all_article_kgs, filepath='/kaggle/working/article_knowledge_graphs.json'):
    """
    Save knowledge graphs to JSON file.
    
    Args:
        all_article_kgs: List of dictionaries containing KG information
        filepath: Path to save the JSON file
    """
    # Convert NetworkX graphs to serializable format
    serializable_kgs = []
    
    for kg_data in all_article_kgs:
        # Convert NetworkX graph to node-link format
        graph = kg_data['graph']
        graph_dict = nx.node_link_data(graph) if graph.number_of_nodes() > 0 else None
        
        serializable_kg = {
            'id': kg_data['id'],
            'text': kg_data['text'],
            'entities': kg_data['entities'],
            'relations': kg_data['relations'],
            'num_nodes': kg_data['num_nodes'],
            'num_edges': kg_data['num_edges'],
            'graph_data': graph_dict  # Serialized graph
        }
        
        serializable_kgs.append(serializable_kg)
    
    # Save to JSON
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(serializable_kgs, f, indent=2, ensure_ascii=False)
    
    print(f"\n{'='*80}")
    print("KNOWLEDGE GRAPHS SAVED")
    print(f"{'='*80}")
    print(f"Saved {len(serializable_kgs)} knowledge graphs to:")
    print(f"  {filepath}")
    print(f"\nFile size: {len(json.dumps(serializable_kgs)) / (1024*1024):.2f} MB")
    print(f"Articles with graphs: {sum(1 for kg in serializable_kgs if kg['num_edges'] > 0)}")
    print(f"Articles without graphs: {sum(1 for kg in serializable_kgs if kg['num_edges'] == 0)}")

# Save the knowledge graphs
save_knowledge_graphs(all_article_kgs, '/kaggle/working/article_knowledge_graphs.json')

print("\n✓ Knowledge graphs saved successfully!")
print("You can now proceed to the topic classification step.")


KNOWLEDGE GRAPHS SAVED
Saved 1811 knowledge graphs to:
  /kaggle/working/article_knowledge_graphs.json

File size: 1.24 MB
Articles with graphs: 1650
Articles without graphs: 161

✓ Knowledge graphs saved successfully!
You can now proceed to the topic classification step.


In [37]:
# ============================================================================
# STEP 3: LOAD DATA AND EXISTING KNOWLEDGE GRAPHS
# ============================================================================

def load_json_data(filepath: str) -> List[Dict]:
    """Load JSON data from file"""
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"✓ Loaded {len(data)} documents from {filepath}")
    return data

def load_knowledge_graphs(filepath: str) -> List[Dict]:
    """Load previously created knowledge graphs"""
    if not os.path.exists(filepath):
        print(f"\n⚠️  WARNING: Knowledge graph file not found at {filepath}")
        return None
    
    with open(filepath, 'r', encoding='utf-8') as f:
        kgs = json.load(f)
    print(f"✓ Loaded {len(kgs)} knowledge graphs from {filepath}")
    return kgs

# Load your data
articles = load_json_data('/kaggle/input/zstikg/Mp3 playerData-set.json')
knowledge_graphs = load_knowledge_graphs('/kaggle/working/article_knowledge_graphs.json')

# Only classify articles that have KGs
articles_with_kgs = articles[:len(knowledge_graphs)] 

if knowledge_graphs is None:
    print("\n❌ Cannot proceed without knowledge graphs. Please save them first.")
else:
    print(f"\n✓ Successfully loaded {len(articles)} articles and {len(knowledge_graphs)} KGs")
    print("✓ Ready to proceed with topic classification!")

✓ Loaded 1811 documents from /kaggle/input/zstikg/Mp3 playerData-set.json
✓ Loaded 1811 knowledge graphs from /kaggle/working/article_knowledge_graphs.json

✓ Successfully loaded 1811 articles and 1811 KGs
✓ Ready to proceed with topic classification!


In [38]:
# ============================================================================
# STEP 4: EXTRACT ALL UNIQUE CONCEPTS FROM DATASET
# ============================================================================

def extract_unique_concepts(articles: List[Dict]) -> List[str]:
    """
    Extract all unique concepts from the dataset's 'Concept' field.
    
    Args:
        articles: List of article dictionaries
    
    Returns:
        Sorted list of unique concepts
    """
    all_concepts = set()
    
    for article in articles:
        # Get concepts from the Concept field
        if 'Concept' in article and article['Concept']:
            if isinstance(article['Concept'], list):
                all_concepts.update([c.lower().strip() for c in article['Concept'] if c])
            elif isinstance(article['Concept'], str):
                all_concepts.add(article['Concept'].lower().strip())
    
    # Convert to sorted list
    unique_concepts = sorted(list(all_concepts))
    
    print(f"\n{'='*80}")
    print(f"UNIQUE CONCEPTS EXTRACTED FROM DATASET")
    print(f"{'='*80}")
    print(f"Total unique concepts: {len(unique_concepts)}")
    print(f"\nFirst 20 concepts: {unique_concepts[:20]}")
    print(f"\nLast 20 concepts: {unique_concepts[-20:]}")
    
    return unique_concepts


# Extract all unique concepts from the dataset
all_concepts = extract_unique_concepts(articles)

# Save concept list for reference
with open('/kaggle/working/all_concepts.json', 'w') as f:
    json.dump(all_concepts, f, indent=2)

print(f"\n✓ Saved concept list to /kaggle/working/all_concepts.json")


UNIQUE CONCEPTS EXTRACTED FROM DATASET
Total unique concepts: 20

First 20 concepts: ['battery', 'button', 'design', 'eax', 'hardware', 'manual', 'navigation', 'player', 'playlist', 'price', 'screen', 'search', 'size', 'software', 'sound', 'storage', 'transfer', 'use', 'warranty', 'weight']

Last 20 concepts: ['battery', 'button', 'design', 'eax', 'hardware', 'manual', 'navigation', 'player', 'playlist', 'price', 'screen', 'search', 'size', 'software', 'sound', 'storage', 'transfer', 'use', 'warranty', 'weight']

✓ Saved concept list to /kaggle/working/all_concepts.json


In [39]:
# ============================================================================
# STEP 5: KG FORMATTING FUNCTION
# ============================================================================

def format_kg_for_llm(kg_data: Dict) -> str:
    """Format knowledge graph into readable text for the LLM."""
    if not kg_data or kg_data.get('num_edges', 0) == 0:
        return "No knowledge graph available."
    
    # Format entities
    entities = kg_data.get('entities', [])
    entities_str = ", ".join(entities[:30]) if entities else "None"
    if len(entities) > 30:
        entities_str += f"... ({len(entities) - 30} more)"
    
    # Format relationships
    relations = kg_data.get('relations', [])
    if relations:
        relationships = []
        for relation in relations[:20]:
            if isinstance(relation, (list, tuple)) and len(relation) >= 3:
                source, rel_type, target = relation[0], relation[1], relation[2]
                relationships.append(f"{source} --[{rel_type}]--> {target}")
        relationships_str = "\n".join(relationships)
        if len(relations) > 20:
            relationships_str += f"\n... ({len(relations) - 20} more relationships)"
    else:
        relationships_str = "None"
    
    kg_summary = f"""Knowledge Graph Summary:
Entities: {entities_str}

Key Relationships:
{relationships_str}"""
    
    return kg_summary

print("✓ KG formatting function ready")

✓ KG formatting function ready


In [41]:
# ============================================================================
# STEP 6: DEFINE DSPY SIGNATURE FOR TOPIC CLASSIFICATION
# ============================================================================

class TopicClassification(Signature):
    """Given an article text, its knowledge graph, and a list of possible topics,
    determine which topics (can be multiple) are most relevant to this article.
    Return only topic names that are actually present in the available_topics list."""
    
    article_text: str = InputField(
        desc="The text content of the article"
    )
    knowledge_graph: str = InputField(
        desc="Knowledge graph extracted from the article showing entities and relationships"
    )
    available_topics: str = InputField(
        desc="Comma-separated list of all possible topic/concept names"
    )
    
    predicted_topics: str = OutputField(
        desc="Comma-separated list of relevant topics from the available_topics list. "
             "Only return topics that actually appear in the available_topics list. "
             "If multiple topics apply, list them all (maximum 5). If no topics match well, return 'none'."
    )
    #confidence: str = OutputField(
        #desc="Confidence level: high, medium, or low"
    #)
    #reasoning: str = OutputField(
        #desc="Brief explanation of why these topics were chosen based on the article and knowledge graph"
    #)

print("✓ Topic classification signature defined")

✓ Topic classification signature defined


In [42]:
# ============================================================================
# STEP 7: CREATE TOPIC CLASSIFIER CLASS
# ============================================================================

class ArticleTopicClassifier:
    """Multi-label topic classifier using article text, KG, and available topics."""
    
    def __init__(self, available_topics: List[str]):
        self.available_topics = available_topics
        self.available_topics_str = ", ".join(available_topics)
        self.classifier = dspy.ChainOfThought(TopicClassification)
        print(f"✓ Classifier initialized with {len(available_topics)} possible topics")
    
    def classify_article(self, article_text: str, kg_data: Dict) -> Dict:
        """Classify a single article."""
        kg_summary = format_kg_for_llm(kg_data)
        
        # Truncate article if too long
        #max_text_length = 500
        #if len(article_text) > max_text_length:
            #article_text = article_text[:max_text_length] + "..."
        
        try:
            result = self.classifier(
                article_text=article_text,
                knowledge_graph=kg_summary,
                available_topics=self.available_topics_str
            )
            
            predicted_topics_raw = result.predicted_topics
            
            if predicted_topics_raw.lower() == 'none':
                predicted_topics = []
            else:
                predicted_topics = [
                    t.strip().lower() 
                    for t in predicted_topics_raw.split(',')
                    if t.strip()
                ]
                predicted_topics = [
                    t for t in predicted_topics 
                    if t in [at.lower() for at in self.available_topics]
                ]
            
            return {
                'predicted_topics': predicted_topics,
                'num_topics': len(predicted_topics),
                #'confidence': result.confidence,
                #'reasoning': result.reasoning
            }
            
        except Exception as e:
            print(f"  Error: {str(e)}")
            return {
                'predicted_topics': [],
                'num_topics': 0,
                #'confidence': 'error',
                #'reasoning': f'Classification failed: {str(e)}'
            }
    
    def classify_dataset(self, articles: List[Dict], knowledge_graphs: List[Dict],
                        max_articles: int = None) -> List[Dict]:
        """Classify multiple articles."""
        results = []
        kg_dict = {kg['id']: kg for kg in knowledge_graphs}
        
        if max_articles:
            articles = articles[:max_articles]
        
        print(f"\n{'='*80}")
        print(f"CLASSIFYING {len(articles)} ARTICLES")
        print(f"{'='*80}\n")
        
        for idx, article in enumerate(articles):
            print(f"Processing article {idx}...", end=" ")
            
            article_text = article.get('Article Text', '')
            kg_data = kg_dict.get(idx, {'num_edges': 0})
            classification = self.classify_article(article_text, kg_data)
            
            result = {
                'article_id': idx,
                'article_text': article_text + "...",
                'true_concepts': article.get('Concept', []),
                'predicted_topics': classification['predicted_topics'],
                'num_predicted': classification['num_topics'],
                #'confidence': classification['confidence'],
                #'reasoning': classification['reasoning']
            }
            
            results.append(result)
            print(f"✓ Found {len(classification['predicted_topics'])} topics")
        
        return results

print("✓ Classifier class defined")

✓ Classifier class defined


In [43]:
# ============================================================================
# STEP 8: ANALYSIS FUNCTIONS
# ============================================================================

def analyze_classification_results(results: List[Dict], available_topics: List[str]) -> Dict:
    """Analyze classification results and generate statistics."""
    from collections import Counter
    
    topic_counts = Counter()
    #confidence_dist = Counter()
    
    multi_label_count = 0
    no_label_count = 0
    
    for result in results:
        predicted = result['predicted_topics']
        
        if len(predicted) == 0:
            no_label_count += 1
        elif len(predicted) > 1:
            multi_label_count += 1
        
        for topic in predicted:
            topic_counts[topic] += 1
        
        #confidence_dist[result['confidence']] += 1
    
    stats = {
        'total_articles': len(results),
        'articles_with_topics': len(results) - no_label_count,
        'articles_without_topics': no_label_count,
        'multi_label_articles': multi_label_count,
        #'avg_topics_per_article': sum(len(r['predicted_topics']) for r in results) / len(results),
        #'most_common_topics': topic_counts.most_common(10)
    }
    
    print(f"\n{'='*80}")
    print("CLASSIFICATION RESULTS SUMMARY")
    print(f"{'='*80}")
    print(f"\nTotal articles: {stats['total_articles']}")
    print(f"Articles with topics: {stats['articles_with_topics']} ({stats['articles_with_topics']/stats['total_articles']*100:.1f}%)")
    print(f"Multi-label articles: {stats['multi_label_articles']} ({stats['multi_label_articles']/stats['total_articles']*100:.1f}%)")
    #print(f"Average topics per article: {stats['avg_topics_per_article']:.2f}")
    
    print(f"\n{'='*80}")
    print("TOP 10 MOST FREQUENT TOPICS")
    print(f"{'='*80}")
    #for topic, count in stats['most_common_topics']:
        #print(f"  {topic}: {count} articles ({count/stats['total_articles']*100:.1f}%)")
    
    return stats

def save_results(results: List[Dict], output_path: str):
    """Save classification results to JSON file."""
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f"\n✓ Results saved to {output_path}")

def export_to_csv(results: List[Dict], output_path: str):
    """Export results to CSV for easy analysis."""
    rows = []
    for r in results:
        rows.append({
            'article_id': r['article_id'],
            #'article_preview': r['article_text'],
            'true_concepts': '|'.join(r['true_concepts']) if r['true_concepts'] else '',
            'predicted_topics': '|'.join(r['predicted_topics']),
            'num_predicted': len(r['predicted_topics']),
            #'confidence': r['confidence'],
            #'reasoning': r['reasoning']
        })
    
    df = pd.DataFrame(rows)
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"✓ CSV exported to {output_path}")

print("✓ Analysis functions defined")

✓ Analysis functions defined


In [44]:
# ============================================================================
# RUN CLASSIFICATION ON ALL THE ARTICLES
# ============================================================================

# Initialize classifier
classifier = ArticleTopicClassifier(all_concepts)

# Classify all the articles with KGs
results = classifier.classify_dataset(
    articles=articles,
    knowledge_graphs=knowledge_graphs,
    max_articles=10000
)

# Analyze results
stats = analyze_classification_results(results, all_concepts)

# Save results
save_results(results, '/kaggle/working/topic_classification_results.json')
export_to_csv(results, '/kaggle/working/topic_classification_results.csv')

# Show sample results
print(f"\n{'='*80}")
print("SAMPLE RESULTS (First 3)")
print(f"{'='*80}\n")

for result in results[:3]:
    print(f"Article {result['article_id']}:")
    print(f"  Preview: {result['article_text']}")
    print(f"  True: {result['true_concepts']}")
    print(f"  Predicted: {result['predicted_topics']}")
    #print(f"  Confidence: {result['confidence']}")
    print()

✓ Classifier initialized with 20 possible topics

CLASSIFYING 1811 ARTICLES

Processing article 0... ✓ Found 3 topics
Processing article 1... ✓ Found 5 topics
Processing article 2... ✓ Found 4 topics
Processing article 3... ✓ Found 1 topics
Processing article 4... ✓ Found 5 topics
Processing article 5... ✓ Found 3 topics
Processing article 6... ✓ Found 1 topics
Processing article 7... ✓ Found 3 topics
Processing article 8... ✓ Found 5 topics
Processing article 9... ✓ Found 2 topics
Processing article 10... ✓ Found 3 topics
Processing article 11... ✓ Found 3 topics
Processing article 12... ✓ Found 1 topics
Processing article 13... ✓ Found 1 topics
Processing article 14... ✓ Found 5 topics
Processing article 15... ✓ Found 5 topics
Processing article 16... ✓ Found 3 topics
Processing article 17... ✓ Found 3 topics
Processing article 18... ✓ Found 4 topics
Processing article 19... ✓ Found 1 topics
Processing article 20... ✓ Found 4 topics
Processing article 21... ✓ Found 2 topics
Processin

In [45]:
# ============================================================================
# EVALUATION: Compare Predicted vs True Concepts
# ============================================================================

def evaluate_classification(results: List[Dict]) -> Dict:
    """
    Calculate accuracy metrics by comparing predicted topics with true concepts.
    """
    total = len(results)
    exact_match = 0  # All predicted topics match exactly with true concepts
    partial_match = 0  # At least one topic matches
    no_match = 0  # No topics match
    no_true_labels = 0  # Articles with no true concepts
    
    all_precisions = []
    all_recalls = []
    all_f1s = []
    
    correct_predictions = []
    incorrect_predictions = []
    
    for result in results:
        true_set = set([c.lower().strip() for c in result['true_concepts']]) if result['true_concepts'] else set()
        pred_set = set(result['predicted_topics'])
        
        # Skip articles with no true labels
        if len(true_set) == 0:
            no_true_labels += 1
            continue
        
        # Find intersection
        intersection = true_set & pred_set
        
        # Exact match
        if true_set == pred_set and len(true_set) > 0:
            exact_match += 1
            correct_predictions.append(result)
        elif len(intersection) > 0:
            partial_match += 1
        else:
            no_match += 1
            incorrect_predictions.append(result)
        
        # Calculate precision, recall, F1
        if len(pred_set) > 0:
            precision = len(intersection) / len(pred_set)
            all_precisions.append(precision)
        else:
            all_precisions.append(0)
        
        if len(true_set) > 0:
            recall = len(intersection) / len(true_set)
            all_recalls.append(recall)
        else:
            all_recalls.append(0)
        
        if all_precisions[-1] + all_recalls[-1] > 0:
            f1 = 2 * (all_precisions[-1] * all_recalls[-1]) / (all_precisions[-1] + all_recalls[-1])
            all_f1s.append(f1)
        else:
            all_f1s.append(0)
    
    articles_with_true_labels = total - no_true_labels
    
    metrics = {
        'total_articles': total,
        'articles_with_true_labels': articles_with_true_labels,
        'articles_without_true_labels': no_true_labels,
        'exact_matches': exact_match,
        'partial_matches': partial_match,
        'no_matches': no_match,
        'exact_match_rate': exact_match / articles_with_true_labels if articles_with_true_labels > 0 else 0,
        'partial_match_rate': partial_match / articles_with_true_labels if articles_with_true_labels > 0 else 0,
        'avg_precision': np.mean(all_precisions) if all_precisions else 0,
        'avg_recall': np.mean(all_recalls) if all_recalls else 0,
        'avg_f1': np.mean(all_f1s) if all_f1s else 0,
        'correct_examples': correct_predictions[:3],
        'incorrect_examples': incorrect_predictions[:3]
    }
    
    # Print results
    print(f"\n{'='*80}")
    print("EVALUATION METRICS")
    print(f"{'='*80}")
    print(f"\nTotal articles evaluated: {articles_with_true_labels}")
    print(f"Articles without labels (skipped): {no_true_labels}")
    print(f"\n📊 ACCURACY:")
    print(f"  ✓ Exact matches: {exact_match} ({metrics['exact_match_rate']*100:.1f}%)")
    print(f"  ~ Partial matches: {partial_match} ({metrics['partial_match_rate']*100:.1f}%)")
    print(f"  ✗ No matches: {no_match} ({no_match/articles_with_true_labels*100:.1f}%)")
    print(f"\n📈 PERFORMANCE METRICS:")
    print(f"  Precision: {metrics['avg_precision']:.3f} (how many predicted topics were correct)")
    print(f"  Recall: {metrics['avg_recall']:.3f} (how many true topics were found)")
    print(f"  F1 Score: {metrics['avg_f1']:.3f} (overall accuracy)")
    
    return metrics


# Run evaluation
print("\nEvaluating classification accuracy...")
eval_metrics = evaluate_classification(results)

# Show correct examples
print(f"\n{'='*80}")
print("✓ CORRECTLY CLASSIFIED EXAMPLES")
print(f"{'='*80}")
for ex in eval_metrics['correct_examples']:
    print(f"\nArticle {ex['article_id']}:")
    print(f"  True: {ex['true_concepts']}")
    print(f"  Predicted: {ex['predicted_topics']}")
    print(f"  ✓ EXACT MATCH!")

# Show incorrect examples
print(f"\n{'='*80}")
print("✗ INCORRECTLY CLASSIFIED EXAMPLES")
print(f"{'='*80}")
for ex in eval_metrics['incorrect_examples']:
    print(f"\nArticle {ex['article_id']}:")
    print(f"  True: {ex['true_concepts']}")
    print(f"  Predicted: {ex['predicted_topics']}")
    #print(f"  Reasoning: {ex['reasoning'][:150]}...")


Evaluating classification accuracy...

EVALUATION METRICS

Total articles evaluated: 1174
Articles without labels (skipped): 637

📊 ACCURACY:
  ✓ Exact matches: 307 (26.1%)
  ~ Partial matches: 615 (52.4%)
  ✗ No matches: 252 (21.5%)

📈 PERFORMANCE METRICS:
  Precision: 0.539 (how many predicted topics were correct)
  Recall: 0.673 (how many true topics were found)
  F1 Score: 0.557 (overall accuracy)

✓ CORRECTLY CLASSIFIED EXAMPLES

Article 13:
  True: ['player']
  Predicted: ['player']
  ✓ EXACT MATCH!

Article 19:
  True: ['software']
  Predicted: ['software']
  ✓ EXACT MATCH!

Article 27:
  True: ['price']
  Predicted: ['price']
  ✓ EXACT MATCH!

✗ INCORRECTLY CLASSIFIED EXAMPLES

Article 1:
  True: ['use']
  Predicted: ['design', 'hardware', 'software', 'sound', 'storage']

Article 8:
  True: ['player']
  Predicted: ['design', 'hardware', 'price', 'sound', 'storage']

Article 16:
  True: ['transfer']
  Predicted: ['player', 'storage', 'use']


In [46]:
# ============================================================================
# EVALUATION METRICS (Following the Paper's Methodology)
# ============================================================================

def calculate_metrics_paper_style(results: List[Dict]) -> Dict:
    """
    Calculate evaluation metrics following the paper's approach:
    "For each article, the model inferred topic(s) were compared against 
    the list of 'gold' topic(s) to compute the true positive, false positive, 
    and false negative statistics for that article. Then, all such statistics 
    for all the articles in a dataset were aggregated and used to compute 
    the final Precision, Recall, and micro-averaged F1 score."
    
    Reference: Section 5.4 of the paper
    """
    # Aggregate statistics across all articles
    total_tp = 0  # True Positives
    total_fp = 0  # False Positives
    total_fn = 0  # False Negatives
    
    articles_evaluated = 0
    articles_skipped = 0
    
    for result in results:
        # Get true and predicted topics (lowercased and stripped)
        true_topics = set([c.lower().strip() for c in result['true_concepts']]) if result['true_concepts'] else set()
        pred_topics = set([t.lower().strip() for t in result['predicted_topics']])
        
        # Skip articles with no ground truth labels
        if len(true_topics) == 0:
            articles_skipped += 1
            continue
        
        articles_evaluated += 1
        
        # Calculate TP, FP, FN for this article
        tp = len(true_topics & pred_topics)  # Intersection
        fp = len(pred_topics - true_topics)  # Predicted but not true
        fn = len(true_topics - pred_topics)  # True but not predicted
        
        # Aggregate
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
    # Calculate micro-averaged metrics
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'total_tp': total_tp,
        'total_fp': total_fp,
        'total_fn': total_fn,
        'articles_evaluated': articles_evaluated,
        'articles_skipped': articles_skipped
    }
    
    # Print results
    print(f"\n{'='*80}")
    print("EVALUATION METRICS (Paper's Methodology)")
    print(f"{'='*80}")
    print(f"\nArticles evaluated: {articles_evaluated}")
    print(f"Articles skipped (no ground truth): {articles_skipped}")
    print(f"\nAggregated Statistics:")
    print(f"  True Positives (TP): {total_tp}")
    print(f"  False Positives (FP): {total_fp}")
    print(f"  False Negatives (FN): {total_fn}")
    print(f"\n📊 MICRO-AVERAGED METRICS:")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall: {recall:.3f}")
    print(f"  F1 Score: {f1_score:.3f}")
    
    return metrics


# Run evaluation using the paper's methodology
print("\nEvaluating using paper's methodology...")
paper_metrics = calculate_metrics_paper_style(results)

# Compare with their baselines (from Table 6 in the paper)
print(f"\n{'='*80}")
print("COMPARISON WITH PAPER'S BASELINES")
print(f"{'='*80}")
print(f"This model's F1 Score: {paper_metrics['f1_score']:.3f}")
print(f"\nPaper's Results:")
print(f"  GFLM-S baseline: 0.532")
print(f"  GFLM-W baseline: 0.530")
print(f"  SBERT (best mid encoder): 0.594")
print(f"  ChatGPT-3.5 (best overall): 0.606")


Evaluating using paper's methodology...

EVALUATION METRICS (Paper's Methodology)

Articles evaluated: 1174
Articles skipped (no ground truth): 637

Aggregated Statistics:
  True Positives (TP): 1128
  False Positives (FP): 1457
  False Negatives (FN): 604

📊 MICRO-AVERAGED METRICS:
  Precision: 0.436
  Recall: 0.651
  F1 Score: 0.523

COMPARISON WITH PAPER'S BASELINES
This model's F1 Score: 0.523

Paper's Results:
  GFLM-S baseline: 0.532
  GFLM-W baseline: 0.530
  SBERT (best mid encoder): 0.594
  ChatGPT-3.5 (best overall): 0.606


In [47]:
# ============================================================================
# STEP 3: LOAD KEYWORDS
# ============================================================================

# Load the keyword file
with open('/kaggle/input/keywords/Keyword_Creative.json', 'r') as f:
    keyword_data = json.load(f)

# Parse into dictionary: {topic_name: [keywords]}
concept_keywords = {}

for item in keyword_data:
    keywords_list = item['Keyword']
    topic_name = keywords_list[0].lower()  # First item is topic name
    related_keywords = [kw.lower() for kw in keywords_list[1:]]  # Rest are keywords
    concept_keywords[topic_name] = related_keywords

print(f"\n{'='*80}")
print("LOADED CONCEPT KEYWORDS")
print(f"{'='*80}")
print(f"Total concepts: {len(concept_keywords)}\n")

# Show first 3 examples
for i, (topic, keywords) in enumerate(list(concept_keywords.items())[:3]):
    print(f"{i+1}. {topic}:")
    print(f"   Keywords: {', '.join(keywords[:5])}{'...' if len(keywords) > 5 else ''}")


LOADED CONCEPT KEYWORDS
Total concepts: 21

1. software:
   Keywords: installation, explorer, folder_structure, program, setup...
2. player:
   Keywords: creative, nomad, zen, xtra
3. sound:
   Keywords: audio, headphone, earbud, earphone, equalizer...


In [48]:
# ============================================================================
# STEP 4: BUILD CONCEPT KNOWLEDGE GRAPHS
# ============================================================================

class ConceptKGBuilder:
    """Build knowledge graphs for concepts using topic names and keywords."""
    
    def __init__(self):
        self.concept_kgs = {}
    
    def build_concept_kg(self, topic_name: str, keywords: List[str]) -> nx.DiGraph:
        """Build a KG for a single concept."""
        G = nx.DiGraph()
        
        # Add topic as central node
        G.add_node(topic_name, node_type='topic')
        
        # Add keywords and connect to topic
        for keyword in keywords:
            G.add_node(keyword, node_type='keyword')
            G.add_edge(topic_name, keyword, relation='has_keyword')
        
        # Connect keywords to each other (co-occurrence)
        for i, kw1 in enumerate(keywords):
            for kw2 in keywords[i+1:]:
                G.add_edge(kw1, kw2, relation='co_occurs')
        
        return G
    
    def build_all_concept_kgs(self, concept_keywords: Dict[str, List[str]]) -> Dict[str, nx.DiGraph]:
        """Build KGs for all concepts."""
        for topic, keywords in concept_keywords.items():
            self.concept_kgs[topic] = self.build_concept_kg(topic, keywords)
        
        print(f"\n{'='*80}")
        print("BUILT CONCEPT KNOWLEDGE GRAPHS")
        print(f"{'='*80}")
        print(f"Total concept KGs: {len(self.concept_kgs)}")
        
        # Show stats
        for topic, kg in list(self.concept_kgs.items())[:3]:
            print(f"  {topic}: {kg.number_of_nodes()} nodes, {kg.number_of_edges()} edges")
        
        return self.concept_kgs

# Build concept KGs
kg_builder = ConceptKGBuilder()
concept_kgs = kg_builder.build_all_concept_kgs(concept_keywords)


BUILT CONCEPT KNOWLEDGE GRAPHS
Total concept KGs: 21
  software: 11 nodes, 55 edges
  player: 5 nodes, 10 edges
  sound: 10 nodes, 45 edges


In [49]:
# ============================================================================
# STEP 5: GRAPH EMBEDDER
# ============================================================================

class GraphEmbedder:
    """Embed graphs using Graph2Vec."""
    
    def __init__(self, dimensions=128, wl_iterations=3):
        """
        Args:
            dimensions: Embedding dimension
            wl_iterations: Weisfeiler-Lehman iterations
        """
        self.dimensions = dimensions
        self.model = Graph2Vec(
            dimensions=dimensions,
            wl_iterations=wl_iterations,
            epochs=100,
            min_count=1
        )
        print(f"\n✓ Initialized Graph2Vec (dim={dimensions}, WL={wl_iterations})")
    
    def embed_graphs(self, graphs: List[nx.Graph]) -> np.ndarray:
        """
        Embed multiple graphs.
        
        Args:
            graphs: List of NetworkX graphs
        
        Returns:
            Embedding matrix (n_graphs × dimensions)
        """
        # Convert to undirected (Graph2Vec requires undirected)
        undirected = [G.to_undirected() if G.is_directed() else G for G in graphs]
        
        # Fit and get embeddings
        self.model.fit(undirected)
        embeddings = self.model.get_embedding()
        
        return embeddings
    
    def embed_single_graph(self, G: nx.Graph, reference_graphs: List[nx.Graph]) -> np.ndarray:
        """
        Embed a single new graph using reference graphs.
        
        Args:
            G: Graph to embed
            reference_graphs: Graphs to fit model on
        
        Returns:
            Single embedding vector
        """
        # Combine reference graphs with new graph
        all_graphs = reference_graphs + [G]
        embeddings = self.embed_graphs(all_graphs)
        
        # Return embedding of the last graph (the new one)
        return embeddings[-1]

print("✓ GraphEmbedder class defined")

✓ GraphEmbedder class defined


In [50]:
# ============================================================================
# STEP 6: GRAPH-BASED TOPIC CLASSIFIER
# ============================================================================

class GraphTopicClassifier:
    """Classify articles using graph embeddings."""
    
    def __init__(self, 
                 concept_kgs: Dict[str, nx.DiGraph],
                 embedding_dim: int = 128,
                 threshold: float = 0.3):
        """
        Initialize classifier.
        
        Args:
            concept_kgs: Dictionary of concept KGs
            embedding_dim: Embedding dimension
            threshold: Similarity threshold for topic assignment
        """
        self.concept_kgs = concept_kgs
        self.threshold = threshold
        
        print(f"\n{'='*80}")
        print("INITIALIZING GRAPH-BASED CLASSIFIER")
        print(f"{'='*80}")
        
        # Initialize embedder
        self.embedder = GraphEmbedder(dimensions=embedding_dim)
        
        # Prepare concept graphs
        self.concept_names = list(concept_kgs.keys())
        concept_graphs = [concept_kgs[name] for name in self.concept_names]
        
        # Embed all concept graphs
        print("\nEmbedding concept knowledge graphs...")
        self.concept_embeddings = self.embedder.embed_graphs(concept_graphs)
        
        print(f"✓ Embedded {len(self.concept_names)} concept graphs")
        print(f"  Embedding shape: {self.concept_embeddings.shape}")
        print(f"  Similarity threshold: {threshold}")
        
        # Store as dictionary
        self.concept_embedding_dict = {
            name: self.concept_embeddings[i] 
            for i, name in enumerate(self.concept_names)
        }
    
    def _dict_to_graph(self, kg_dict: Dict) -> nx.DiGraph:
        """Convert KG dictionary to NetworkX graph."""
        G = nx.DiGraph()
        
        # Add nodes from entities
        entities = kg_dict.get('entities', [])
        for entity in entities:
            G.add_node(entity)
        
        # Add edges from relations
        relations = kg_dict.get('relations', [])
        for relation in relations:
            if isinstance(relation, (list, tuple)) and len(relation) >= 3:
                source, rel_type, target = relation[0], relation[1], relation[2]
                G.add_edge(source, target, relation=rel_type)
        
        return G
    
    def classify_article(self, article_kg: Dict) -> Dict:
        """Classify a single article."""
        # Convert to graph
        G_article = self._dict_to_graph(article_kg)
        
        # Skip if empty
        if G_article.number_of_nodes() == 0:
            return {
                'predicted_topics': [],
                'similarities': {},
                'max_similarity': 0.0
            }
        
        # Embed article graph
        reference_graphs = [self.concept_kgs[name] for name in self.concept_names]
        article_embedding = self.embedder.embed_single_graph(
            G_article, 
            reference_graphs=reference_graphs
        )
        
        # Compute similarities with all concepts
        similarities = {}
        for topic_name in self.concept_names:
            concept_emb = self.concept_embedding_dict[topic_name]
            sim = cosine_similarity([article_embedding], [concept_emb])[0][0]
            similarities[topic_name] = float(sim)
        
        # Assign topics above threshold
        predicted_topics = [
            topic for topic, sim in similarities.items() 
            if sim >= self.threshold
        ]
        
        # Sort by similarity
        predicted_topics = sorted(
            predicted_topics, 
            key=lambda t: similarities[t], 
            reverse=True
        )
        
        return {
            'predicted_topics': predicted_topics,
            'similarities': similarities,
            'max_similarity': max(similarities.values()) if similarities else 0.0
        }
    
    def classify_dataset(self, 
                        articles: List[Dict],
                        knowledge_graphs: List[Dict]) -> List[Dict]:
        """Classify all articles."""
        results = []
        kg_dict = {kg['id']: kg for kg in knowledge_graphs}
        
        print(f"\n{'='*80}")
        print(f"CLASSIFYING {len(knowledge_graphs)} ARTICLES")
        print(f"{'='*80}\n")
        
        for idx in range(len(knowledge_graphs)):
            if idx % 20 == 0:
                print(f"Processing articles {idx}-{min(idx+20, len(knowledge_graphs))}...")
            
            article = articles[idx] if idx < len(articles) else {}
            article_kg = kg_dict.get(idx, {'entities': [], 'relations': []})
            
            # Skip if no KG
            if article_kg.get('num_edges', 0) == 0:
                results.append({
                    'article_id': idx,
                    'predicted_topics': [],
                    'similarities': {},
                    'max_similarity': 0.0,
                    'true_concepts': article.get('Concept', [])
                })
                continue
            
            # Classify
            classification = self.classify_article(article_kg)
            
            result = {
                'article_id': idx,
                'predicted_topics': classification['predicted_topics'],
                'similarities': classification['similarities'],
                'max_similarity': classification['max_similarity'],
                'true_concepts': article.get('Concept', [])
            }
            
            results.append(result)
        
        print("\n✓ Classification complete!")
        return results

print("✓ GraphTopicClassifier class defined")

✓ GraphTopicClassifier class defined


In [51]:
# ============================================================================
# STEP 6: GRAPH-BASED TOPIC CLASSIFIER
# ============================================================================

class GraphTopicClassifier:
    """Classify articles using graph embeddings."""
    
    def __init__(self, 
                 concept_kgs: Dict[str, nx.DiGraph],
                 embedding_dim: int = 128,
                 threshold: float = 0.3):
        """
        Initialize classifier.
        
        Args:
            concept_kgs: Dictionary of concept KGs
            embedding_dim: Embedding dimension
            threshold: Similarity threshold for topic assignment
        """
        self.concept_kgs = concept_kgs
        self.threshold = threshold
        
        print(f"\n{'='*80}")
        print("INITIALIZING GRAPH-BASED CLASSIFIER")
        print(f"{'='*80}")
        
        # Initialize embedder
        self.embedder = GraphEmbedder(dimensions=embedding_dim)
        
        # Prepare concept graphs
        self.concept_names = list(concept_kgs.keys())
        concept_graphs = [concept_kgs[name] for name in self.concept_names]
        
        # Embed all concept graphs
        print("\nEmbedding concept knowledge graphs...")
        self.concept_embeddings = self.embedder.embed_graphs(concept_graphs)
        
        print(f"✓ Embedded {len(self.concept_names)} concept graphs")
        print(f"  Embedding shape: {self.concept_embeddings.shape}")
        print(f"  Similarity threshold: {threshold}")
        
        # Store as dictionary
        self.concept_embedding_dict = {
            name: self.concept_embeddings[i] 
            for i, name in enumerate(self.concept_names)
        }
    
    def _dict_to_graph(self, kg_dict: Dict) -> nx.DiGraph:
        """Convert KG dictionary to NetworkX graph."""
        G = nx.DiGraph()
        
        # Add nodes from entities
        entities = kg_dict.get('entities', [])
        for entity in entities:
            G.add_node(entity)
        
        # Add edges from relations
        relations = kg_dict.get('relations', [])
        for relation in relations:
            if isinstance(relation, (list, tuple)) and len(relation) >= 3:
                source, rel_type, target = relation[0], relation[1], relation[2]
                G.add_edge(source, target, relation=rel_type)
        
        return G
    
    def classify_article(self, article_kg: Dict) -> Dict:
        """Classify a single article."""
        # Convert to graph
        G_article = self._dict_to_graph(article_kg)
        
        # Skip if empty
        if G_article.number_of_nodes() == 0:
            return {
                'predicted_topics': [],
                'similarities': {},
                'max_similarity': 0.0
            }
        
        # Embed article graph
        reference_graphs = [self.concept_kgs[name] for name in self.concept_names]
        article_embedding = self.embedder.embed_single_graph(
            G_article, 
            reference_graphs=reference_graphs
        )
        
        # Compute similarities with all concepts
        similarities = {}
        for topic_name in self.concept_names:
            concept_emb = self.concept_embedding_dict[topic_name]
            sim = cosine_similarity([article_embedding], [concept_emb])[0][0]
            similarities[topic_name] = float(sim)
        
        # Assign topics above threshold
        predicted_topics = [
            topic for topic, sim in similarities.items() 
            if sim >= self.threshold
        ]
        
        # Sort by similarity
        predicted_topics = sorted(
            predicted_topics, 
            key=lambda t: similarities[t], 
            reverse=True
        )
        
        return {
            'predicted_topics': predicted_topics,
            'similarities': similarities,
            'max_similarity': max(similarities.values()) if similarities else 0.0
        }
    
    def classify_dataset(self, 
                        articles: List[Dict],
                        knowledge_graphs: List[Dict]) -> List[Dict]:
        """Classify all articles."""
        results = []
        kg_dict = {kg['id']: kg for kg in knowledge_graphs}
        
        print(f"\n{'='*80}")
        print(f"CLASSIFYING {len(knowledge_graphs)} ARTICLES")
        print(f"{'='*80}\n")
        
        for idx in range(len(knowledge_graphs)):
            if idx % 20 == 0:
                print(f"Processing articles {idx}-{min(idx+20, len(knowledge_graphs))}...")
            
            article = articles[idx] if idx < len(articles) else {}
            article_kg = kg_dict.get(idx, {'entities': [], 'relations': []})
            
            # Skip if no KG
            if article_kg.get('num_edges', 0) == 0:
                results.append({
                    'article_id': idx,
                    'predicted_topics': [],
                    'similarities': {},
                    'max_similarity': 0.0,
                    'true_concepts': article.get('Concept', [])
                })
                continue
            
            # Classify
            classification = self.classify_article(article_kg)
            
            result = {
                'article_id': idx,
                'predicted_topics': classification['predicted_topics'],
                'similarities': classification['similarities'],
                'max_similarity': classification['max_similarity'],
                'true_concepts': article.get('Concept', [])
            }
            
            results.append(result)
        
        print("\n✓ Classification complete!")
        return results

print("✓ GraphTopicClassifier class defined")

✓ GraphTopicClassifier class defined


In [52]:
!pip uninstall -y node2vec gensim smart_open
!pip install smart_open==5.2.1 --force-reinstall
!pip install gensim==4.3.2 --force-reinstall
!pip install node2vec==0.4.6 --force-reinstall

Found existing installation: node2vec 0.4.6
Uninstalling node2vec-0.4.6:
  Successfully uninstalled node2vec-0.4.6
Found existing installation: gensim 4.4.0
Uninstalling gensim-4.4.0:
  Successfully uninstalled gensim-4.4.0
Found existing installation: smart-open 5.2.1
Uninstalling smart-open-5.2.1:
  Successfully uninstalled smart-open-5.2.1
  Using cached smart_open-5.2.1-py3-none-any.whl.metadata (22 kB)
Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Using cached gensim-4.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.3 kB)
  Using cached numpy-2.3.4-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.16.3-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached smart_open-7.4.4-py3-none-any.whl.metadata (24 kB)
Using cached gensim-4.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-2.3.4-cp311-cp311-manylinux_2_27_x86_64.m

In [53]:
!pip install smart_open==5.2.1 --force-reinstall

  Using cached smart_open-5.2.1-py3-none-any.whl.metadata (22 kB)
Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Attempting uninstall: smart_open
    Found existing installation: smart_open 7.4.4
    Uninstalling smart_open-7.4.4:
      Successfully uninstalled smart_open-7.4.4


In [ ]:
import os, sys
os._exit(0)

In [54]:
import json
import numpy as np
import networkx as nx
from typing import List, Dict
from node2vec import Node2Vec
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

print("✓ Starting Multi-Label Article Classifier\n")

✓ Starting Multi-Label Article Classifier



In [55]:
# ============================================================================
# STEP 1: GRAPH EMBEDDER
# ============================================================================

class GraphEmbedder:
    """Embed graphs using Node2Vec."""
    
    def __init__(self, dimensions=128, walk_length=30, num_walks=200):
        self.dimensions = dimensions
        self.walk_length = walk_length
        self.num_walks = num_walks
        print(f"✓ Initialized Node2Vec embedder (dim={dimensions})")
    
    def embed_graph(self, G: nx.Graph) -> np.ndarray:
        """Embed a single graph."""
        # Convert to undirected
        G_undirected = G.to_undirected() if G.is_directed() else G
        
        # Skip if too small
        if G_undirected.number_of_nodes() < 2:
            return np.zeros(self.dimensions)
        
        try:
            # Fit Node2Vec
            node2vec = Node2Vec(
                G_undirected, 
                dimensions=self.dimensions,
                walk_length=self.walk_length,
                num_walks=self.num_walks,
                workers=1,
                quiet=True
            )
            
            model = node2vec.fit(window=10, min_count=1, batch_words=4)
            
            # Get embeddings for all nodes
            node_embeddings = []
            for node in G_undirected.nodes():
                node_embeddings.append(model.wv[str(node)])
            
            # Average all node embeddings to get graph embedding
            graph_embedding = np.mean(node_embeddings, axis=0)
            return graph_embedding
        except:
            return np.zeros(self.dimensions)
    
    def embed_graphs(self, graphs: List[nx.Graph]) -> np.ndarray:
        """Embed multiple graphs."""
        embeddings = []
        for i, G in enumerate(graphs):
            if i % 5 == 0:
                print(f"  Embedding graph {i+1}/{len(graphs)}...")
            emb = self.embed_graph(G)
            embeddings.append(emb)
        return np.array(embeddings)

In [56]:
# ============================================================================
# STEP 2: MULTI-LABEL GRAPH CLASSIFIER
# ============================================================================

class MultiLabelGraphClassifier:
    """Multi-label classifier using graph embeddings."""
    
    def __init__(self, 
                 concept_kgs: Dict[str, nx.DiGraph],
                 embedding_dim: int = 128,
                 threshold: float = 0.5,  # Increased from 0.3
                 top_k: int = 3):  # Max labels per article
        """
        Initialize classifier.
        
        Args:
            concept_kgs: Dictionary of concept knowledge graphs
            embedding_dim: Embedding dimension
            threshold: Minimum similarity threshold (increased to 0.5)
            top_k: Maximum number of topics to assign per article
        """
        self.concept_kgs = concept_kgs
        self.threshold = threshold
        self.top_k = top_k
        
        print(f"\n{'='*80}")
        print("INITIALIZING MULTI-LABEL GRAPH CLASSIFIER")
        print(f"{'='*80}")
        
        # Initialize embedder
        self.embedder = GraphEmbedder(dimensions=embedding_dim)
        
        # Prepare concept graphs
        self.concept_names = list(concept_kgs.keys())
        concept_graphs = [concept_kgs[name] for name in self.concept_names]
        
        # Embed all concept graphs
        print("\nEmbedding concept knowledge graphs...")
        self.concept_embeddings = self.embedder.embed_graphs(concept_graphs)
        
        print(f"\n✓ Embedded {len(self.concept_names)} concept graphs")
        print(f"  Embedding shape: {self.concept_embeddings.shape}")
        print(f"  Absolute threshold: {threshold}")
        print(f"  Top-K limit: {self.top_k}")
        print(f"  Relative gap threshold: 0.15")
        
        # Store as dictionary
        self.concept_embedding_dict = {
            name: self.concept_embeddings[i] 
            for i, name in enumerate(self.concept_names)
        }
    
    def _dict_to_graph(self, kg_dict: Dict) -> nx.DiGraph:
        """Convert KG dictionary to NetworkX graph."""
        G = nx.DiGraph()
        
        # Add nodes from entities
        entities = kg_dict.get('entities', [])
        for entity in entities:
            G.add_node(entity)
        
        # Add edges from relations
        relations = kg_dict.get('relations', [])
        for relation in relations:
            if isinstance(relation, (list, tuple)) and len(relation) >= 3:
                source, rel_type, target = relation[0], relation[1], relation[2]
                G.add_edge(source, target, relation=rel_type)
        
        return G
    
    def classify_article(self, article_kg: Dict, top_k: int = 3) -> Dict:
        """
        Classify a single article (multi-label).
        
        Strategy: Use adaptive thresholding to prevent overprediction.
        - Apply absolute threshold
        - Take top-K only
        - Use relative threshold (gap from max)
        
        Args:
            article_kg: Article knowledge graph dictionary
            top_k: Maximum number of labels to assign
        """
        # Convert to graph
        G_article = self._dict_to_graph(article_kg)
        
        # Skip if empty
        if G_article.number_of_nodes() == 0:
            return {
                'predicted_topics': [],
                'similarities': {},
                'max_similarity': 0.0
            }
        
        # Embed article graph
        article_embedding = self.embedder.embed_graph(G_article)
        
        # Compute similarities with all concepts
        similarities = {}
        for topic_name in self.concept_names:
            concept_emb = self.concept_embedding_dict[topic_name]
            sim = cosine_similarity([article_embedding], [concept_emb])[0][0]
            similarities[topic_name] = float(sim)
        
        # Get sorted topics by similarity
        sorted_topics = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
        
        # Strategy 1: Absolute threshold
        candidates = [(topic, sim) for topic, sim in sorted_topics if sim >= self.threshold]
        
        if len(candidates) == 0:
            return {
                'predicted_topics': [],
                'similarities': similarities,
                'max_similarity': max(similarities.values()) if similarities else 0.0
            }
        
        # Strategy 2: Top-K limit (prevent overprediction)
        candidates = candidates[:top_k]
        
        # Strategy 3: Relative threshold (gap detection)
        # Only keep topics within 0.15 similarity of the best match
        max_sim = candidates[0][1]
        relative_threshold = max_sim - 0.35
        
        predicted_topics = [
            topic for topic, sim in candidates
            if sim >= relative_threshold
        ]
        
        return {
            'predicted_topics': predicted_topics,
            'similarities': similarities,
            'max_similarity': max(similarities.values()) if similarities else 0.0
        }
    
    def classify_dataset(self, 
                        articles: List[Dict],
                        knowledge_graphs: List[Dict]) -> List[Dict]:
        """Classify all articles."""
        results = []
        kg_dict = {kg['id']: kg for kg in knowledge_graphs}
        
        print(f"\n{'='*80}")
        print(f"CLASSIFYING {len(knowledge_graphs)} ARTICLES (MULTI-LABEL)")
        print(f"{'='*80}\n")
        
        for idx in range(len(knowledge_graphs)):
            if idx % 20 == 0:
                print(f"Processing articles {idx}-{min(idx+20, len(knowledge_graphs))}...")
            
            article = articles[idx] if idx < len(articles) else {}
            article_kg = kg_dict.get(idx, {'entities': [], 'relations': []})
            
            # Skip if no KG
            if article_kg.get('num_edges', 0) == 0:
                results.append({
                    'article_id': idx,
                    'predicted_topics': [],
                    'similarities': {},
                    'max_similarity': 0.0,
                    'true_concepts': article.get('Concept', [])
                })
                continue
            
            # Classify (multi-label)
            classification = self.classify_article(article_kg)
            
            result = {
                'article_id': idx,
                'predicted_topics': classification['predicted_topics'],
                'num_predicted': len(classification['predicted_topics']),
                'similarities': classification['similarities'],
                'max_similarity': classification['max_similarity'],
                'true_concepts': article.get('Concept', [])
            }
            
            results.append(result)
        
        print("\n✓ Classification complete!")
        return results

In [57]:
# ============================================================================
# STEP 3: EVALUATION METRICS
# ============================================================================

def calculate_metrics(results: List[Dict]) -> Dict:
    """
    Calculate multi-label classification metrics.
    Uses micro-averaged precision, recall, F1.
    """
    total_tp = 0
    total_fp = 0
    total_fn = 0
    
    articles_evaluated = 0
    articles_skipped = 0
    
    multi_label_count = 0
    single_label_count = 0
    no_label_count = 0
    
    for result in results:
        true_topics = set([c.lower().strip() for c in result['true_concepts']]) if result['true_concepts'] else set()
        pred_topics = set([t.lower().strip() for t in result['predicted_topics']])
        
        if len(true_topics) == 0:
            articles_skipped += 1
            continue
        
        articles_evaluated += 1
        
        # Count label distribution
        if len(pred_topics) == 0:
            no_label_count += 1
        elif len(pred_topics) == 1:
            single_label_count += 1
        else:
            multi_label_count += 1
        
        tp = len(true_topics & pred_topics)
        fp = len(pred_topics - true_topics)
        fn = len(true_topics - pred_topics)
        
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'total_tp': total_tp,
        'total_fp': total_fp,
        'total_fn': total_fn,
        'articles_evaluated': articles_evaluated,
        'articles_skipped': articles_skipped,
        'multi_label_count': multi_label_count,
        'single_label_count': single_label_count,
        'no_label_count': no_label_count
    }
    
    print(f"\n{'='*80}")
    print("EVALUATION METRICS (Micro-Averaged)")
    print(f"{'='*80}")
    print(f"\nArticles evaluated: {articles_evaluated}")
    print(f"Articles skipped (no ground truth): {articles_skipped}")
    print(f"\nLabel Distribution:")
    print(f"  No labels: {no_label_count} ({no_label_count/articles_evaluated*100:.1f}%)")
    print(f"  Single label: {single_label_count} ({single_label_count/articles_evaluated*100:.1f}%)")
    print(f"  Multi-label: {multi_label_count} ({multi_label_count/articles_evaluated*100:.1f}%)")
    print(f"\nAggregated Statistics:")
    print(f"  True Positives (TP): {total_tp}")
    print(f"  False Positives (FP): {total_fp}")
    print(f"  False Negatives (FN): {total_fn}")
    print(f"\n📊 PERFORMANCE:")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall: {recall:.3f}")
    print(f"  F1 Score: {f1_score:.3f}")
    
    # Comparison with baselines
    print(f"\n{'='*80}")
    print("COMPARISON WITH PAPER BASELINES (Medical Dataset)")
    print(f"{'='*80}")
    print(f"  This model:           F1 = {f1_score:.3f}")
    print(f"  GFLM-S baseline:      F1 = 0.532")
    print(f"  GFLM-W baseline:      F1 = 0.530")
    print(f"  SBERT:                F1 = 0.594")
    print(f"  ChatGPT-3.5:          F1 = 0.606")
    
    return metrics


In [58]:
# ============================================================================
# STEP 4: RUN CLASSIFICATION
# ============================================================================

print("\n" + "="*80)
print("RUNNING MULTI-LABEL CLASSIFICATION")
print("="*80)

# Initialize classifier with concept KGs
classifier = MultiLabelGraphClassifier(
    concept_kgs=concept_kgs,
    embedding_dim=128,
    threshold=0.5,  # Higher threshold = fewer predictions
    top_k=5  # Max 3 labels per article
)

# Classify all articles
results = classifier.classify_dataset(
    articles=articles,
    knowledge_graphs=knowledge_graphs
)

# Calculate metrics
metrics = calculate_metrics(results)


RUNNING MULTI-LABEL CLASSIFICATION

INITIALIZING MULTI-LABEL GRAPH CLASSIFIER
✓ Initialized Node2Vec embedder (dim=128)

Embedding concept knowledge graphs...
  Embedding graph 1/21...
  Embedding graph 6/21...
  Embedding graph 11/21...
  Embedding graph 16/21...
  Embedding graph 21/21...

✓ Embedded 21 concept graphs
  Embedding shape: (21, 128)
  Absolute threshold: 0.5
  Top-K limit: 3
  Relative gap threshold: 0.15

CLASSIFYING 1811 ARTICLES (MULTI-LABEL)

Processing articles 0-20...
Processing articles 20-40...
Processing articles 40-60...
Processing articles 60-80...
Processing articles 80-100...
Processing articles 100-120...
Processing articles 120-140...
Processing articles 140-160...
Processing articles 160-180...
Processing articles 180-200...
Processing articles 200-220...
Processing articles 220-240...
Processing articles 240-260...
Processing articles 260-280...
Processing articles 280-300...
Processing articles 300-320...
Processing articles 320-340...
Processing arti

In [60]:
# ============================================================================
# THREE APPROACHES TO TOPIC EMBEDDING (FIXED VERSION)
# ============================================================================

!pip install sentence-transformers -q

import json
import numpy as np
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

print("✓ Imports complete\n")


class SimpleKGClassifier:
    """Classify articles by comparing KG embeddings to topic embeddings."""
    
    def __init__(self, 
                 topic_names: List[str],
                 topic_keywords: Dict[str, List[str]] = None,
                 topic_embedding_method: str = 'name_only',
                 threshold: float = 0.3):
        """
        Initialize classifier.
        
        Args:
            topic_names: List of topic names
            topic_keywords: Dictionary mapping topics to keywords
            topic_embedding_method: 'name_only', 'name_plus_keywords', or 'average_embeddings'
            threshold: Similarity threshold
        """
        self.topic_names = [t.lower() for t in topic_names]
        self.topic_keywords = topic_keywords
        self.method = topic_embedding_method
        self.threshold = threshold
        
        # Initialize SBERT model
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        
        print(f"\n{'='*80}")
        print(f"INITIALIZING CLASSIFIER - METHOD: {topic_embedding_method.upper()}")
        print(f"{'='*80}")
        print(f"Total topics: {len(self.topic_names)}")
        print(f"Threshold: {threshold}\n")
        
        # Embed topics based on method
        print(f"Embedding topics using method: {topic_embedding_method}...")
        
        if topic_embedding_method == 'name_only':
            self.topic_embeddings = self._embed_method1_name_only()
        elif topic_embedding_method == 'name_plus_keywords':
            self.topic_embeddings = self._embed_method2_name_plus_keywords()
        elif topic_embedding_method == 'average_embeddings':
            self.topic_embeddings = self._embed_method3_average_embeddings()
        else:
            raise ValueError(f"Unknown method: {topic_embedding_method}")
        
        print(f"✓ Embedded {len(self.topic_names)} topics")
        print(f"  Embedding dimension: {self.topic_embeddings.shape[1]}")
    
    def _embed_method1_name_only(self) -> np.ndarray:
        """Method 1: Embed only topic names."""
        print("  Method 1: Topic name only")
        print(f"  Example: 'phone' → embedding")
        
        embeddings = self.model.encode(self.topic_names)
        return embeddings
    
    def _embed_method2_name_plus_keywords(self) -> np.ndarray:
        """Method 2: Embed topic + keywords as concatenated text."""
        print("  Method 2: Topic + keywords as text")
        
        topic_texts = []
        for topic in self.topic_names:
            if self.topic_keywords and topic in self.topic_keywords and self.topic_keywords[topic]:
                keywords = ", ".join(self.topic_keywords[topic][:10])
                topic_text = f"{topic}: {keywords}"
            else:
                topic_text = topic
            topic_texts.append(topic_text)
        
        print(f"  Example: '{topic_texts[0][:80]}...'")
        
        embeddings = self.model.encode(topic_texts)
        return embeddings
    
    def _embed_method3_average_embeddings(self) -> np.ndarray:
        """Method 3: Average embeddings of topic name + keywords."""
        print("  Method 3: Average of topic and keyword embeddings")
        
        all_embeddings = []
        
        for topic in self.topic_names:
            # Embed topic name
            topic_emb = self.model.encode(topic)
            
            # Embed keywords if available
            if self.topic_keywords and topic in self.topic_keywords and self.topic_keywords[topic]:
                keywords = self.topic_keywords[topic][:10]
                keyword_embs = self.model.encode(keywords)
                
                # Average topic + keyword embeddings
                all_embs = np.vstack([topic_emb.reshape(1, -1), keyword_embs])
                avg_emb = np.mean(all_embs, axis=0)
            else:
                # No keywords - just use topic embedding
                avg_emb = topic_emb
            
            all_embeddings.append(avg_emb)
        
        print(f"  Example: avg(embed('phone'), embed('nokia'), embed('quality'))")
        
        return np.array(all_embeddings)
    
    def kg_to_text(self, kg_dict: Dict) -> str:
        """Convert KG to text representation."""
        entities = kg_dict.get('entities', [])
        
        if not entities:
            return ""
        
        # Just list entities
        text = ", ".join(entities[:50])
        return text
    
    def classify_article(self, article_kg: Dict) -> Dict:
        """Classify a single article."""
        kg_text = self.kg_to_text(article_kg)
        
        if not kg_text:
            return {
                'predicted_topics': [],
                'similarities': {},
                'max_similarity': 0.0
            }
        
        # Embed KG
        kg_embedding = self.model.encode(kg_text)
        
        # Compute similarities
        similarities = {}
        for i, topic_name in enumerate(self.topic_names):
            sim = cosine_similarity(
                [kg_embedding], 
                [self.topic_embeddings[i]]
            )[0][0]
            similarities[topic_name] = float(sim)
        
        # Assign topics above threshold
        predicted_topics = [
            topic for topic, sim in similarities.items() 
            if sim >= self.threshold
        ]
        
        predicted_topics = sorted(
            predicted_topics, 
            key=lambda t: similarities[t], 
            reverse=True
        )
        
        return {
            'predicted_topics': predicted_topics,
            'similarities': similarities,
            'max_similarity': max(similarities.values()) if similarities else 0.0
        }
    
    def classify_dataset(self, 
                        articles: List[Dict],
                        knowledge_graphs: List[Dict]) -> List[Dict]:
        """Classify all articles."""
        results = []
        kg_dict = {kg['id']: kg for kg in knowledge_graphs}
        
        print(f"\n{'='*80}")
        print(f"CLASSIFYING {len(knowledge_graphs)} ARTICLES")
        print(f"{'='*80}\n")
        
        for idx in range(len(knowledge_graphs)):
            if idx % 20 == 0:
                print(f"Processing articles {idx}-{min(idx+20, len(knowledge_graphs))}...")
            
            article = articles[idx] if idx < len(articles) else {}
            article_kg = kg_dict.get(idx, {'entities': [], 'relations': []})
            
            classification = self.classify_article(article_kg)
            
            result = {
                'article_id': idx,
                'predicted_topics': classification['predicted_topics'],
                'similarities': classification['similarities'],
                'max_similarity': classification['max_similarity'],
                'true_concepts': article.get('Concept', [])
            }
            
            results.append(result)
        
        print("\n✓ Classification complete!")
        return results

✓ Imports complete



In [61]:
# ============================================================================
# TEST ALL THREE METHODS
# ============================================================================

# Get topic names
all_topic_names = list(concept_keywords.keys())

methods = {
    'name_only': 'Method 1: Topic Name Only',
    'name_plus_keywords': 'Method 2: Topic + Keywords as Text',
    'average_embeddings': 'Method 3: Average Embeddings'
}

results_comparison = {}

for method_key, method_name in methods.items():
    print(f"\n\n{'#'*80}")
    print(f"TESTING: {method_name}")
    print(f"{'#'*80}")
    
    # Initialize classifier
    classifier = SimpleKGClassifier(
        topic_names=all_topic_names,
        topic_keywords=concept_keywords,
        topic_embedding_method=method_key,
        threshold=0.3
    )
    
    # Classify
    results = classifier.classify_dataset(
        articles=articles,
        knowledge_graphs=knowledge_graphs
    )
    
    # Evaluate
    metrics = calculate_metrics_paper_style(results)
    
    # Store results
    results_comparison[method_key] = {
        'metrics': metrics,
        'results': results
    }
    
    # Save
    save_results(results, f'/kaggle/working/{method_key}_results.json')
    export_to_csv(results, f'/kaggle/working/{method_key}_results.csv')




################################################################################
TESTING: Method 1: Topic Name Only
################################################################################

INITIALIZING CLASSIFIER - METHOD: NAME_ONLY
Total topics: 21
Threshold: 0.3

Embedding topics using method: name_only...
  Method 1: Topic name only
  Example: 'phone' → embedding


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✓ Embedded 21 topics
  Embedding dimension: 384

CLASSIFYING 1811 ARTICLES

Processing articles 0-20...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


✓ Classification complete!

EVALUATION METRICS (Paper's Methodology)

Articles evaluated: 1174
Articles skipped (no ground truth): 637

Aggregated Statistics:
  True Positives (TP): 690
  False Positives (FP): 1934
  False Negatives (FN): 1042

📊 MICRO-AVERAGED METRICS:
  Precision: 0.263
  Recall: 0.398
  F1 Score: 0.317

✓ Results saved to /kaggle/working/average_embeddings_results.json
✓ CSV exported to /kaggle/working/average_embeddings_results.csv


In [63]:
# ============================================================================
# FINAL COMPARISON OF ALL METHODS
# ============================================================================

print(f"\n\n{'='*80}")
print("FINAL COMPARISON: ALL METHODS")
print(f"{'='*80}\n")

comparison_df = pd.DataFrame({
    'Method': [
        'Method 1: Name Only',
        'Method 2: Name + Keywords Text',
        'Method 3: Average Embeddings',
        'LLM Reasoning (baseline)',
        'Paper ChatGPT (baseline)'
    ],
    'F1 Score': [
        results_comparison['name_only']['metrics']['f1_score'],
        results_comparison['name_plus_keywords']['metrics']['f1_score'],
        results_comparison['average_embeddings']['metrics']['f1_score'],
        0.818,
        0.606
    ],
    'Precision': [
        results_comparison['name_only']['metrics']['precision'],
        results_comparison['name_plus_keywords']['metrics']['precision'],
        results_comparison['average_embeddings']['metrics']['precision'],
        0.719,
        '-'
    ],
    'Recall': [
        results_comparison['name_only']['metrics']['recall'],
        results_comparison['name_plus_keywords']['metrics']['recall'],
        results_comparison['average_embeddings']['metrics']['recall'],
        0.948,
        '-'
    ]
})

print(comparison_df.to_string(index=False))

# Find best method
best_method = max(results_comparison.items(), key=lambda x: x[1]['metrics']['f1_score'])
print(f"\n{'='*80}")
print(f"🏆 BEST METHOD: {methods[best_method[0]]}")
print(f"   F1 Score: {best_method[1]['metrics']['f1_score']:.3f}")
print(f"{'='*80}")



FINAL COMPARISON: ALL METHODS

                        Method  F1 Score Precision    Recall
           Method 1: Name Only  0.280011  0.262785  0.299654
Method 2: Name + Keywords Text  0.420282  0.396217   0.44746
  Method 3: Average Embeddings  0.316804  0.262957  0.398383
      LLM Reasoning (baseline)  0.818000     0.719     0.948
      Paper ChatGPT (baseline)  0.606000         -         -

🏆 BEST METHOD: Method 2: Topic + Keywords as Text
   F1 Score: 0.420
